# Prep any indices for HydroTas 2020-2021 workplan:
- Skill assessment
  - rainfall, surface temp and surface wind over Australia region and Tasmania region
  - Assess skill as function of start month and ensemble size
- UNSEEN
  - Tasmanian rainfall and Melbourne surface temperature

In [1]:
import xarray as xr
import myfuncs as my
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

# Cluster only needed for ocean files

In [2]:
cluster = SLURMCluster(
    walltime='01:00:00', 
    cores=6, 
    memory='36GB', 
    job_extra=['--qos="express"'])

In [3]:
cluster.scale(jobs=6)
client = Client(cluster)
client

Client Scheduler: tcp://10.141.0.138:35665 Dashboard: http://10.141.0.138:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


# Surface winds

In [4]:
REGIONS = ['AUS', 'MEL', 'TAS']

In [3]:
def wind_speed(u,v):
    return xr.ufuncs.sqrt(u ** 2 + v ** 2).to_dataset(name='V_ref')

In [4]:
for region in REGIONS:
    # f6
    ds = my.open_mfzarr(
        [f'data/f6_u_ref_{region}_raw.zarr',
         f'data/f6_v_ref_{region}_raw.zarr'])
    V_ref = wind_speed(ds['u_ref'], ds['v_ref'])
    my.to_zarr(V_ref, f'data/f6_V_ref_{region}_raw.zarr')
    
    # f5
    ds = my.open_mfzarr(
        [f'data/f5_u_ref_{region}_raw.zarr',
         f'data/f5_v_ref_{region}_raw.zarr'])
    V_ref = wind_speed(ds['u_ref'], ds['v_ref'])
    my.to_zarr(V_ref, f'data/f5_V_ref_{region}_raw.zarr')
    
    # JRA55
    ds = my.open_mfzarr(
        [f'data/jra55_u_ref_{region}.zarr',
         f'data/jra55_v_ref_{region}.zarr'])
    V_ref = wind_speed(ds['u_ref'], ds['v_ref'])
    my.to_zarr(V_ref, f'data/jra55_V_ref_{region}.zarr')
    
    # JRA55
    ds = my.open_mfzarr(
        [f'data/jra55_u_ref_{region}_ts.zarr',
         f'data/jra55_v_ref_{region}_ts.zarr'])
    V_ref = wind_speed(ds['u_ref'], ds['v_ref'])
    my.to_zarr(V_ref, f'data/jra55_V_ref_{region}_ts.zarr')

# NINO3.4

In [12]:
def latlon_average(ds, box, wrap_lons=True, lat_name='lat', lon_name='lon'):
    def _get_latlon_region(ds, box, wrap_lons=True):
        ds = ds.assign_coords({lon_name: (ds[lon_name] + 360)  % 360})
        lon_inds = np.where(
            np.logical_and(ds[lon_name].values>=box[2], 
                           ds[lon_name].values<=box[3]))[0]
        lat_inds = np.where(
            np.logical_and(ds[lat_name].values>=box[0], 
                           ds[lat_name].values<=box[1]))[0]
        return ds.isel({lon_name: lon_inds, lat_name: lat_inds})
    
    return _get_latlon_region(ds, box, wrap_lons).mean(dim=[lat_name, lon_name])
    
def nino34(sst_anom, wrap_lons=True, lat_name='lat', lon_name='lon'):
    box = [-5.0,5.0,190.0,240.0]
        
    return latlon_average(sst_anom, box, wrap_lons, lat_name, lon_name)

In [13]:
sst = my.open_zarr(f'data/f6_sst_NATIVE_raw.zarr.zip')
nino34 = nino34(sst, lat_name='yt_ocean', lon_name='xt_ocean').rename({'sst':'nino34'})
my.to_zarr(nino34, f'data/f6_nino34_raw.zarr')

# DMI

In [20]:
def dmi(sst_anom, wrap_lons=True, lat_name='lat', lon_name='lon'):
    boxW = [-10.0,10.0,50.0,70.0]
    boxE = [-10.0,0.0,90.0,110.0]
    
    da_W = latlon_average(sst_anom, boxW, wrap_lons, lat_name, lon_name)
    da_E = latlon_average(sst_anom, boxE, wrap_lons, lat_name, lon_name)
    
    return (da_W - da_E)

In [21]:
sst = my.open_zarr(f'data/f6_sst_NATIVE_raw.zarr.zip')
dmi = dmi(sst, lat_name='yt_ocean', lon_name='xt_ocean').rename({'sst':'dmi'})
my.to_zarr(dmi, f'data/f6_dmi_raw.zarr')

/scratch1/squ027/.conda/envs/pangeo/lib/python3.9/site-packages/xarray/core/indexing.py:1385: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/scratch1/squ027/.conda/envs/pangeo/lib/python3.9/site-packages/xarray/core/indexing.py:1385: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexe

In [22]:
cluster.close()
client.close()